# Введение #

В уроках 2 и 3 мы рассматривали прогнозирование как простую задачу регрессии, где все признаки получаются из одного входа — временного индекса. Мы могли легко строить прогнозы на любой момент в будущем, просто генерируя нужные признаки тренда и сезонности.

Однако когда мы добавили лаговые признаки в уроке 4, характер задачи изменился. Лаговые признаки требуют, чтобы лагированное значение цели было известно на момент прогноза. Лаг 1 сдвигает ряд вперёд на 1 шаг, что означает: можно прогнозировать на 1 шаг вперёд, но не на 2.

В уроке 4 мы просто предполагали, что всегда можем сгенерировать лаги на нужный период прогноза (то есть каждый прогноз был на один шаг вперёд). В реальном прогнозировании обычно требуется больше, поэтому в этом уроке мы научимся строить прогнозы для разных ситуаций.

# Определение задачи прогнозирования #

Перед тем как проектировать модель прогнозирования, нужно определить две вещи:
- какая информация доступна на момент прогноза (признаки), и
- на какой период времени нужны прогнозные значения (цель).

**Forecast origin** — момент времени, в который мы делаем прогноз. На практике его можно считать последним моментом, для которого есть обучающие данные для прогнозируемого ряда. Всё до origin можно использовать для создания признаков.

**Forecast horizon** — период времени, для которого мы делаем прогноз. Часто горизонт описывают количеством шагов: «прогноз на 1 шаг» или «на 5 шагов», например. Горизонт определяет цель.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/xwEgcOk.png" width=500, alt="">
<figcaption style="textalign: center; font-style: italic"><center>Трёхшаговый горизонт прогноза с двухшаговым опережением (lead time) и четырьмя лаговыми признаками. Рисунок показывает одну строку обучающих данных — то есть данные для одного прогноза.
</center></figcaption>
</figure>

Период между origin и horizon — это **lead time** (иногда *latency*) прогноза. Lead time описывается числом шагов от origin до горизонта: «прогноз на 1 шаг вперёд» или «на 3 шага вперёд», например. На практике прогноз может начинаться через несколько шагов после origin из‑за задержек в получении или обработке данных.

# Подготовка данных для прогнозирования #

Чтобы прогнозировать временные ряды с помощью ML‑алгоритмов, нужно преобразовать ряд в датафрейм, который можно использовать с этими алгоритмами. (Если, конечно, вы не используете только детерминированные признаки вроде тренда и сезонности.)

Первую половину этого процесса мы увидели в уроке 4, когда создавали набор признаков из лагов. Вторая половина — подготовка цели. Способ зависит от задачи прогнозирования.

Каждая строка в датафрейме представляет один прогноз. Временной индекс строки — это первый момент горизонта прогноза, но значения всего горизонта размещаются в той же строке. Для многшаговых прогнозов это означает, что модель должна выдавать несколько выходов — по одному на каждый шаг.


In [ ]:

import numpy as np
import pandas as pd

N = 20
ts = pd.Series(
    np.arange(N),
    index=pd.period_range(start='2010', freq='A', periods=N, name='Year'),
    dtype=pd.Int8Dtype,
)

# Lag features
X = pd.DataFrame({
    'y_lag_2': ts.shift(2),
    'y_lag_3': ts.shift(3),
    'y_lag_4': ts.shift(4),
    'y_lag_5': ts.shift(5),
    'y_lag_6': ts.shift(6),    
})

# Multistep targets
y = pd.DataFrame({
    'y_step_3': ts.shift(-2),
    'y_step_2': ts.shift(-1),
    'y_step_1': ts,
})

data = pd.concat({'Targets': y, 'Features': X}, axis=1)

data.head(10).style.set_properties(['Targets'], **{'background-color': 'LavenderBlush'}) \
                   .set_properties(['Features'], **{'background-color': 'Lavender'})

Выше показано, как был бы подготовлен набор данных, аналогичный рисунку *Defining a Forecast*: трёхшаговая задача прогнозирования с двухшаговым lead time и пятью лаговыми признаками. Исходный временной ряд — `y_step_1`. Пропущенные значения можно либо заполнить, либо удалить.

In [ ]:

from pathlib import Path
from warnings import simplefilter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

simplefilter("ignore")

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 4))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)
%config InlineBackend.figure_format = 'retina'


def plot_multistep(y, every=1, ax=None, palette_kwargs=None):
    palette_kwargs_ = dict(palette='husl', n_colors=16, desat=None)
    if palette_kwargs is not None:
        palette_kwargs_.update(palette_kwargs)
    palette = sns.color_palette(**palette_kwargs_)
    if ax is None:
        fig, ax = plt.subplots()
    ax.set_prop_cycle(plt.cycler('color', palette))
    for date, preds in y[::every].iterrows():
        preds.index = pd.period_range(start=date, periods=len(preds))
        preds.plot(ax=ax)
    return ax


data_dir = Path("../input/ts-course-data")
flu_trends = pd.read_csv(data_dir / "flu-trends.csv")
flu_trends.set_index(
    pd.PeriodIndex(flu_trends.Week, freq="W"),
    inplace=True,
)
flu_trends.drop("Week", axis=1, inplace=True)

# Стратегии многшагового прогнозирования #

Существует несколько стратегий для получения нескольких целевых шагов, требуемых прогнозом. Мы рассмотрим четыре распространённые стратегии, каждая со своими плюсами и минусами.

### Multioutput‑модель

Используйте модель, которая естественным образом выдаёт несколько выходов. Линейная регрессия и нейросети умеют это делать. Стратегия простая и эффективная, но доступна не для всех алгоритмов (например, XGBoost так не умеет).

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/uFsHiqr.png" width=300, alt="">
<figcaption style="textalign: center; font-style: italic"><center>
</center></figcaption>
</figure>

### Direct‑стратегия

Обучите отдельную модель для каждого шага горизонта: одна модель прогнозирует на 1 шаг вперёд, другая — на 2 шага и т. д. Прогноз на 1 шаг — другая задача, чем на 2 шага (и т. д.), поэтому полезно иметь отдельную модель для каждого шага. Минус — обучение большого числа моделей может быть вычислительно дорогим.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/HkolNMV.png" width=900, alt="">
<figcaption style="textalign: center; font-style: italic"><center>
</center></figcaption>
</figure>

### Recursive‑стратегия

Обучите одну одношаговую модель и используйте её прогнозы для обновления лаговых признаков на следующий шаг. В рекурсивном подходе мы подаём 1‑шаговый прогноз обратно в ту же модель как лаг для следующего шага. Нужна всего одна модель, но ошибки будут накапливаться по шагам, поэтому прогнозы для длинных горизонтов могут быть неточными.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/sqkSFDn.png" width=300, alt="">
<figcaption style="textalign: center; font-style: italic"><center>
</center></figcaption>
</figure>

### DirRec‑стратегия

Комбинация Direct и Recursive: обучаем модель для каждого шага и используем прогнозы предыдущих шагов как *новые* лаговые признаки. Шаг за шагом каждая модель получает дополнительный лаговый вход. Поскольку у каждой модели всегда актуальные лаги, DirRec может лучше захватывать последовательную зависимость, чем Direct, но, как и Recursive, может страдать от накопления ошибок.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/B7KAvAO.png" width=900, alt="">
<figcaption style="textalign: center; font-style: italic"><center>
</center></figcaption>
</figure>

# Пример — Flu Trends #

В этом примере мы применим стратегии MultiOutput и Direct к данным *Flu Trends* из урока 4, но теперь будем строить реальные прогнозы на несколько недель вперёд относительно обучающего периода.

Определим задачу прогнозирования: горизонт 8 недель и lead time 1 неделя. Иными словами, мы прогнозируем восемь недель случаев гриппа, начиная со следующей недели.

Скрытая ячейка настраивает пример и определяет вспомогательную функцию `plot_multistep`.

In [ ]:
def make_lags(ts, lags, lead_time=1):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(lead_time, lags + lead_time)
        },
        axis=1)


# Four weeks of lag features
y = flu_trends.FluVisits.copy()
X = make_lags(y, lags=4).fillna(0.0)


def make_multistep_target(ts, steps):
    return pd.concat(
        {f'y_step_{i + 1}': ts.shift(-i)
         for i in range(steps)},
        axis=1)


# Eight-week forecast
y = make_multistep_target(y, steps=8).dropna()

# Shifting has created indexes that don't match. Only keep times for
# which we have both targets and features.
y, X = y.align(X, join='inner', axis=0)

Сначала подготовим наш целевой ряд (еженедельные визиты к врачу по поводу гриппа) для многшагового прогнозирования. После этого обучение и прогнозирование будут очень прямолинейными.

In [ ]:

# Create splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

model = LinearRegression()
model.fit(X_train, y_train)

y_fit = pd.DataFrame(model.predict(X_train), index=X_train.index, columns=y.columns)
y_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)

### Multioutput‑модель

Используем линейную регрессию как стратегию MultiOutput. После подготовки данных для нескольких выходов обучение и прогнозирование идут как обычно.

In [ ]:

train_rmse = mean_squared_error(y_train, y_fit, squared=False)
test_rmse = mean_squared_error(y_test, y_pred, squared=False)
print((f"Train RMSE: {train_rmse:.2f}\n" f"Test RMSE: {test_rmse:.2f}"))

palette = dict(palette='husl', n_colors=64)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(11, 6))
ax1 = flu_trends.FluVisits[y_fit.index].plot(**plot_params, ax=ax1)
ax1 = plot_multistep(y_fit, ax=ax1, palette_kwargs=palette)
_ = ax1.legend(['FluVisits (train)', 'Forecast'])
ax2 = flu_trends.FluVisits[y_pred.index].plot(**plot_params, ax=ax2)
ax2 = plot_multistep(y_pred, ax=ax2, palette_kwargs=palette)
_ = ax2.legend(['FluVisits (test)', 'Forecast'])

Помните, что многшаговая модель выдаёт полный прогноз для каждого примера, используемого на вход. В обучающем наборе 269 недель, в тестовом — 90 недель, и теперь для каждой из этих недель у нас есть 8‑шаговый прогноз.

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

model = MultiOutputRegressor(XGBRegressor())
model.fit(X_train, y_train)

y_fit = pd.DataFrame(model.predict(X_train), index=X_train.index, columns=y.columns)
y_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)

### Direct‑стратегия

XGBoost не умеет выдавать несколько выходов в задачах регрессии. Но применив стратегию Direct, мы всё равно можем использовать его для многшаговых прогнозов. Это так же просто, как обернуть его в `MultiOutputRegressor` из scikit‑learn.

In [ ]:

train_rmse = mean_squared_error(y_train, y_fit, squared=False)
test_rmse = mean_squared_error(y_test, y_pred, squared=False)
print((f"Train RMSE: {train_rmse:.2f}\n" f"Test RMSE: {test_rmse:.2f}"))

palette = dict(palette='husl', n_colors=64)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(11, 6))
ax1 = flu_trends.FluVisits[y_fit.index].plot(**plot_params, ax=ax1)
ax1 = plot_multistep(y_fit, ax=ax1, palette_kwargs=palette)
_ = ax1.legend(['FluVisits (train)', 'Forecast'])
ax2 = flu_trends.FluVisits[y_pred.index].plot(**plot_params, ax=ax2)
ax2 = plot_multistep(y_pred, ax=ax2, palette_kwargs=palette)
_ = ax2.legend(['FluVisits (test)', 'Forecast'])

Здесь XGBoost явно переобучается на обучающем наборе. Но на тесте он, похоже, смог уловить часть динамики сезона гриппа лучше, чем линейная регрессия. Вероятно, с подбором гиперпараметров он будет ещё лучше.

Чтобы использовать стратегию DirRec, достаточно заменить `MultiOutputRegressor` на другой обёрточный класс scikit‑learn — `RegressorChain`. Стратегию Recursive нам нужно было бы реализовать самостоятельно.

# Ваш ход #

[**Создайте датасет для прогнозирования**](https://www.kaggle.com/kernels/fork/20667477) для *Store Sales* и примените стратегию DirRec.

---




*Есть вопросы или комментарии? Посетите [форум обсуждений курса](https://www.kaggle.com/learn/time-series/discussion), чтобы пообщаться с другими учащимися.*